# Notebook para teste do projeto QuietWatch (T3BB5)

Este notebook implementa um cliente MQTT responsável por controlar o projeto QuietWatch. O objetivo é automatizar o procedimento de teste e fornecer um tour pela jornada básica de nosso projeto.



## Setup inicial

Primeiramente, é necessário instalar a biblioteca Paho para instanciar um cliente MQTT. Uma vez instalada a biblioteca, *essa bloco não precisa ser executado novamente*.

In [1]:
pip install paho-mqtt

Agora, vamos criar varíaveis com as credenciais de acesso MQTT. Uma vez criadas as variáveis, *esse bloco não precisa ser executado novamente*.

In [2]:
user = "grupo1-bancadaB5"
passwd = "L@Bdygy1B5"

Agora, vamos instanciar nosso objeto cliente MQTT. No momento da conexão também iremos realizar a operação de subscribe nos tópicos de saída **S0**(alerta_mov) e **S1**(calibrando). Observe que a função **on_message** define nossa reação ao receber mensagens nesses tópicos. A partir dessas reações, poderemos verificar os resultados de nossos testes com a saída de nossos códigos. *Esse bloco não precisa ser executado novamente*.

In [3]:
import paho.mqtt.client as mqtt
import time
from datetime import datetime

broker = "3.141.193.238"       
port = 80            
keepAlive = 60
topico_alerta = user+"/S0"
topico_calibrando = user+"/S1"

def on_connect(client, userdata, flags, rc):
    print("Conectado com codigo " + str(rc))
    client.subscribe(topico_alerta, qos=0)
    client.subscribe(topico_calibrando, qos=0)

def on_message(client, userdata, msg):
    topico = msg.topic
    valor = str(msg.payload)
    if(topico.endswith("0")): #topico S0
      if(valor.endswith("1")): #Alerta de Movimento ativado
        now = datetime.now()
        horario = now.strftime("%H:%M:%S")
        print("ALERTA DE MOVIMENTO " + horario)
    else: #topico S1
      if(valor.endswith("1")): #Calibrando ativado
        now = datetime.now()
        horario = now.strftime("%H:%M:%S")
        print("Calibrando... " + horario)

client = mqtt.Client()                      
client.on_connect = on_connect              
client.on_message = on_message              
client.username_pw_set(user, passwd)        
client.connect(broker, port, keepAlive)

0

# Procedimento de Teste

Após o setup inicial, estamos pronto para publicar comandos nos tópicos de entrada **E0**(reset), **E1**(ligar) e **E4**(mode). O projeto QuietWatch possui dois modos de operação: **em casa** e **fora de casa**. Vamos testar cada um deles.

### Testando modo "em casa"

No modo em casa, ao ativar o *alerta_mov*, o movimento do servomotor não é interrompido. Como o usuário se encontra presente na residência, movimento é esperado, dessa forma os eventos são simplesmente registrados para análise posterior.

In [4]:
client.loop_start()
#primeiro vamos garantir que o valor inicial das entradas eh zero
client.publish(user+"/E0", payload="0", qos=0, retain=False)
client.publish(user+"/E1", payload="0", qos=0, retain=False)
client.publish(user+"/E4", payload="0", qos=0, retain=False) #modo em casa
time.sleep(1)
client.publish(user+"/E0", payload="1", qos=0, retain=False) #operação reset
time.sleep(1)
client.publish(user+"/E0", payload="0", qos=0, retain=False)
client.publish(user+"/E1", payload="1", qos=0, retain=False) #operação ligar
time.sleep(30) #tempo para monitorar resultados
client.loop_stop()

Conectado com codigo 0


Ao final desse bloco de código, o circuito está operando no modo **em casa** e temos 30 segundos para interagir com a montagem física enquanto o Python monitora os tópicos de saída do circuito.

### Testando modo "fora de casa"

No modo fora de casa, ao ativar o *alerta_mov*, o movimento do servomotor é interrompido. O objetivo é monitorar a posição onde foi detectado o movimento e apenas retomar a rotina de movimentação normal após a medida do sensor HC-SR04 voltar para o normal.

In [3]:
client.loop_start()
#primeiro vamos garantir que o valor inicial das entradas eh zero
client.publish(user+"/E0", payload="0", qos=0, retain=False)
client.publish(user+"/E1", payload="0", qos=0, retain=False)
client.publish(user+"/E4", payload="0", qos=0, retain=False)
time.sleep(1)
client.publish(user+"/E0", payload="1", qos=0, retain=False) #operação reset
client.publish(user+"/E4", payload="1", qos=0, retain=False) #modo fora de casa
time.sleep(1)
client.publish(user+"/E0", payload="0", qos=0, retain=False)
client.publish(user+"/E1", payload="1", qos=0, retain=False) #operação ligar
time.sleep(30) #tempo para monitorar resultados
client.loop_stop()

Ao final desse bloco de código, o circuito está operando no modo **fora de casa** e temos 30 segundos para interagir com a montagem física enquanto o Python monitora os tópicos de saída do circuito.